- [counter.clsp](counter.clsp)
- [singleton_terminal.clsp](singleton_terminal.clsp)
- [singleton_helpers_v1_1.py](../singleton_helpers_v1_1.py)
- [singleton_top_layer_v1_1.py](https://github.com/Chia-Network/chia-blockchain/blob/1.4.0/chia/wallet/puzzles/singleton_top_layer_v1_1.py)
- [singleton_top_layer_v1_1.clvm](https://github.com/Chia-Network/chia-blockchain/blob/1.4.0/chia/wallet/puzzles/singleton_top_layer_v1_1.clvm)

In [1]:
%%bash
chia version
cdv --version

1.4.0
cdv, version 1.0.8


In [2]:
from pathlib import Path
import sys
sys.path.insert(0, "..")
import singleton_helpers_v1_1

from chia.types.blockchain_format.program import Program
from clvm.casts import int_to_bytes
from clvm_tools.binutils import disassemble
from clvm_tools.clvmc import compile_clvm_text

def load_program(file_path, search_paths):
    clsp = Path(file_path).read_text()
    return Program(
        compile_clvm_text(clsp, search_paths)
    )

def print_program(program):
    print(disassemble(program))

In [3]:
counter_puzzle = load_program("counter.clsp", ".")
singleton_terminal_puzzle =  load_program("singleton_terminal.clsp", ".")
print(counter_puzzle.get_tree_hash())
print(singleton_terminal_puzzle.get_tree_hash())

d5b0cadac424ef077dc8eacc449d0bb8544639cc39f5462a48a8d6b5750a4483
efafe67593029270221b07866b7e4b8dec7226ebd6be177ed4c9051c9ce1cc96


In [4]:
from chia.wallet.puzzles import (singleton_top_layer_v1_1, p2_delegated_puzzle_or_hidden_puzzle)
from cdv.test import Network, Wallet

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice)

print(f'alice balance:\t\t{alice.balance()}')
print(f'alice puzzle hash:\t{alice.puzzle_hash}')

alice balance:		2000000000000
alice puzzle hash:	4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3


In [5]:
PUZZLE_HASH = alice.puzzle_hash
AMOUNT = 1023
MAX_VALUE = 3
VALUE = 0

singleton_terminal_puzzle = singleton_terminal_puzzle.curry(PUZZLE_HASH, AMOUNT)
singleton_counter_puzzle = counter_puzzle.curry(counter_puzzle, MAX_VALUE, singleton_terminal_puzzle, AMOUNT, VALUE)
print(singleton_terminal_puzzle.get_tree_hash())
print(singleton_counter_puzzle.get_tree_hash())

a870977635c6fb664f8fe9147e86e794ffb39a4b6e0ced1c4417a42d94249481
5105569e016af878e23bf86a02ccb75fcae1d61d6e6f0268282355b7af071dc4


In [6]:
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

inner_puzzle = singleton_counter_puzzle
standard_coin_message, coin_spends = singleton_helpers_v1_1.get_create_singleton_coin_spends(
                    standard_txn_coin,
                    standard_txn_puzzle,
                    odd_amount = 1023, 
                    inner_puzzle = inner_puzzle,
                    keys_values = [
                        ("counter", "one")
                    ]
              )
standard_txn_coin_spend = coin_spends[0]
launcher_coin_spend = coin_spends[1]
print("\nstarting coin spend:")
singleton_helpers_v1_1.print_json(standard_txn_coin_spend.to_json_dict())
print("\nlauncher coin spend:")
singleton_helpers_v1_1.print_json(launcher_coin_spend.to_json_dict())
print(f'\nlauncher id: {launcher_coin_spend.coin.name()}')
print(f'\nstandard_coin_message: {standard_coin_message.hex()}')


starting coin spend:
{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff8203ff80ffff3dffa0

In [7]:
# sign the standard txn spend
from blspy import AugSchemeMPL, PrivateKey
from chia.consensus.default_constants import DEFAULT_CONSTANTS

synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)

signature = AugSchemeMPL.sign(synthetic_sk,standard_coin_message)

# push txn to create an eve singleton
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    coin_spends,
    signature
)
await network.push_tx(spend_bundle)

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1749999998977),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>, amount=1023),
  Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: c3dff2bf57480d788070ec0e8c9bb111692cdcd751d3dae351d5747205956e47>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c06

In [8]:
# spend eve singleton
network.sim.pass_blocks(1)
launcher_id = launcher_coin_spend.coin.name()
            
singleton_struct = (
        singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
    )

singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
            singleton_struct,
            inner_puzzle,
    )

singleton_coin = await singleton_helpers_v1_1.get_unspent_singleton(
    get_coin_records_by_parent_ids, 
    launcher_id)
print(singleton_coin)

lineage_proof = singleton_top_layer_v1_1.lineage_proof_for_coinsol(launcher_coin_spend)

inner_solution = Program.to([])

singleton_coin_spend = singleton_helpers_v1_1.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
    )

print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}')
print(f'\nsingleton_puzzle:\n{singleton_puzzle.get_tree_hash()}')

print('\nsingleton_coin_spend:')
singleton_helpers_v1_1.print_json(singleton_coin_spend.to_json_dict())
from blspy import G2Element
spend_bundle = SpendBundle(
    [singleton_coin_spend],
    G2Element()
)
singleton_helpers_v1_1.print_json(spend_bundle.to_json_dict())
print_program(singleton_puzzle)
await network.push_tx(spend_bundle)

{'amount': 1023,
 'parent_coin_info': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570',
 'puzzle_hash': '0xc3dff2bf57480d788070ec0e8c9bb111692cdcd751d3dae351d5747205956e47'}
singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570',
 'puzzle_hash': '0xc3dff2bf57480d788070ec0e8c9bb111692cdcd751d3dae351d5747205956e47'}

lineage_proof:
{'amount': 1023,
 'inner_puzzle_hash': None,
 'parent_name': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba'}

inner_solution:
80

singleton_puzzle:
c3dff2bf57480d788070ec0e8c9bb111692cdcd751d3dae351d5747205956e47

singleton_coin_spend:
{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570",
        "puzzle_hash": "0xc3dff2bf57480d788070ec0e8c9bb111692cdcd751d3dae351d5747205956e47"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff18ff2fff3480ffff01ff04ffff0

{'additions': [Coin(parent_coin_info=<bytes32: 815ecd83433d842c12a2bca4ec08a9d8c795661a12abc42b5b11339a04c0ad16>, puzzle_hash=<bytes32: 57b547a80acabe515c7d41757d1a69b6120d133423781cf70e6f38a364aad244>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: c3dff2bf57480d788070ec0e8c9bb111692cdcd751d3dae351d5747205956e47>, amount=1023)]}

In [9]:
# spend next singleton
v = VALUE + 1
network.sim.pass_blocks(1)
singleton_struct = (
        singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
    )

singleton_counter_puzzle = counter_puzzle.curry(counter_puzzle, MAX_VALUE, singleton_terminal_puzzle, AMOUNT, v)

inner_puzzle = singleton_counter_puzzle
singleton_puzzle: Program = singleton_top_layer_v1_1.puzzle_for_singleton(
    launcher_id,
    inner_puzzle
)

singleton_coin = await singleton_helpers_v1_1.get_unspent_singleton(
    get_coin_records_by_parent_ids, 
    launcher_id)

lineage_proof = singleton_top_layer_v1_1.lineage_proof_for_coinsol(singleton_coin_spend)

inner_solution = Program.to([])

singleton_coin_spend = singleton_helpers_v1_1.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
)

print(f'singleton_struct:\n{singleton_struct}')
print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}')

print('\nsingleton_coin_spend:')
singleton_helpers_v1_1.print_json(singleton_coin_spend.to_json_dict())
from blspy import G2Element
spend_bundle = SpendBundle(
    [singleton_coin_spend],
    G2Element()
)
singleton_helpers_v1_1.print_json(spend_bundle.to_json_dict())
print_program(singleton_puzzle)
await network.push_tx(spend_bundle)

singleton_struct:
(<bytes32: 7faa3253bfddd1e0decb0906b2dc6247bbc4cf608f58345d173adb63e8b47c9f>, (<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, <bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>))
singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0x815ecd83433d842c12a2bca4ec08a9d8c795661a12abc42b5b11339a04c0ad16',
 'puzzle_hash': '0x57b547a80acabe515c7d41757d1a69b6120d133423781cf70e6f38a364aad244'}

lineage_proof:
{'amount': 1023,
 'inner_puzzle_hash': '0x5105569e016af878e23bf86a02ccb75fcae1d61d6e6f0268282355b7af071dc4',
 'parent_name': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570'}

inner_solution:
80

singleton_coin_spend:
{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0x815ecd83433d842c12a2bca4ec08a9d8c795661a12abc42b5b11339a04c0ad16",
        "puzzle_hash": "0x57b547a80acabe515c7d41757d1a69b6120d133423781cf70e6f38a364aad244"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff0

{'additions': [Coin(parent_coin_info=<bytes32: 9f4a5da9066c5d6c1c091d94a01e8138693f969247173c616e2910cfe4ca72b9>, puzzle_hash=<bytes32: 789db1ea2983bbf724a14d5236e272889c0d4ab13e214a16a3659f71be29174c>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: 815ecd83433d842c12a2bca4ec08a9d8c795661a12abc42b5b11339a04c0ad16>, puzzle_hash=<bytes32: 57b547a80acabe515c7d41757d1a69b6120d133423781cf70e6f38a364aad244>, amount=1023)]}

In [10]:
# spend next singleton
v = VALUE + 2
network.sim.pass_blocks(1)
singleton_struct = (
        singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
    )

singleton_counter_puzzle = counter_puzzle.curry(counter_puzzle, MAX_VALUE, singleton_terminal_puzzle, AMOUNT, v)

inner_puzzle = singleton_counter_puzzle
singleton_puzzle: Program = singleton_top_layer_v1_1.puzzle_for_singleton(
    launcher_id,
    inner_puzzle
)

singleton_coin = await singleton_helpers_v1_1.get_unspent_singleton(
    get_coin_records_by_parent_ids, 
    launcher_id)

lineage_proof = singleton_top_layer_v1_1.lineage_proof_for_coinsol(singleton_coin_spend)

inner_solution = Program.to([])

singleton_coin_spend = singleton_helpers_v1_1.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
)

print(f'singleton_struct:\n{singleton_struct}')
print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}')

print('\nsingleton_coin_spend:')
singleton_helpers_v1_1.print_json(singleton_coin_spend.to_json_dict())
from blspy import G2Element
spend_bundle = SpendBundle(
    [singleton_coin_spend],
    G2Element()
)
singleton_helpers_v1_1.print_json(spend_bundle.to_json_dict())
print_program(singleton_puzzle)
await network.push_tx(spend_bundle)

singleton_struct:
(<bytes32: 7faa3253bfddd1e0decb0906b2dc6247bbc4cf608f58345d173adb63e8b47c9f>, (<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, <bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>))
singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0x9f4a5da9066c5d6c1c091d94a01e8138693f969247173c616e2910cfe4ca72b9',
 'puzzle_hash': '0x789db1ea2983bbf724a14d5236e272889c0d4ab13e214a16a3659f71be29174c'}

lineage_proof:
{'amount': 1023,
 'inner_puzzle_hash': '0xfbe3666aa80a296db1b2e93a27eb6e36364528271c0e3f7e45f54a6eca0b83f1',
 'parent_name': '0x815ecd83433d842c12a2bca4ec08a9d8c795661a12abc42b5b11339a04c0ad16'}

inner_solution:
80

singleton_coin_spend:
{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0x9f4a5da9066c5d6c1c091d94a01e8138693f969247173c616e2910cfe4ca72b9",
        "puzzle_hash": "0x789db1ea2983bbf724a14d5236e272889c0d4ab13e214a16a3659f71be29174c"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff0

{'additions': [Coin(parent_coin_info=<bytes32: 4238e577dc4aabb5f8aff3e8ff2695123e712ac7c3670b573e150e7e03e3107b>, puzzle_hash=<bytes32: aaf3a437c916cffd74edbf605cd3d550bc1a0969f9857a212d72ed40e193a084>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: 9f4a5da9066c5d6c1c091d94a01e8138693f969247173c616e2910cfe4ca72b9>, puzzle_hash=<bytes32: 789db1ea2983bbf724a14d5236e272889c0d4ab13e214a16a3659f71be29174c>, amount=1023)]}

In [13]:
# spend next singleton
v = VALUE + 3
network.sim.pass_blocks(1)
singleton_struct = (
        singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
    )

singleton_counter_puzzle = counter_puzzle.curry(counter_puzzle, MAX_VALUE, singleton_terminal_puzzle, AMOUNT, v)

inner_puzzle = singleton_counter_puzzle
singleton_puzzle: Program = singleton_top_layer_v1_1.puzzle_for_singleton(
    launcher_id,
    inner_puzzle
)

singleton_coin = await singleton_helpers_v1_1.get_unspent_singleton(
    get_coin_records_by_parent_ids, 
    launcher_id)

lineage_proof = singleton_top_layer_v1_1.lineage_proof_for_coinsol(singleton_coin_spend)

inner_solution = Program.to([])

singleton_coin_spend = singleton_helpers_v1_1.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
)

print(f'singleton_struct:\n{singleton_struct}')
print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}')

print('\nsingleton_coin_spend:')
singleton_helpers_v1_1.print_json(singleton_coin_spend.to_json_dict())
from blspy import G2Element
spend_bundle = SpendBundle(
    [singleton_coin_spend],
    G2Element()
)
singleton_helpers_v1_1.print_json(spend_bundle.to_json_dict())
print_program(singleton_puzzle)
await network.push_tx(spend_bundle)

singleton_struct:
(<bytes32: 7faa3253bfddd1e0decb0906b2dc6247bbc4cf608f58345d173adb63e8b47c9f>, (<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, <bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>))
singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0x4238e577dc4aabb5f8aff3e8ff2695123e712ac7c3670b573e150e7e03e3107b',
 'puzzle_hash': '0xaaf3a437c916cffd74edbf605cd3d550bc1a0969f9857a212d72ed40e193a084'}

lineage_proof:
{'amount': 1023,
 'inner_puzzle_hash': '0x82549a5c359964fe8cc7b313e6ec7adf2f9c4af8d217dcbd95910eae453c6036',
 'parent_name': '0x4238e577dc4aabb5f8aff3e8ff2695123e712ac7c3670b573e150e7e03e3107b'}

inner_solution:
80

singleton_coin_spend:
{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0x4238e577dc4aabb5f8aff3e8ff2695123e712ac7c3670b573e150e7e03e3107b",
        "puzzle_hash": "0xaaf3a437c916cffd74edbf605cd3d550bc1a0969f9857a212d72ed40e193a084"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff0

{'error': 'Err.ASSERT_MY_PARENT_ID_FAILED'}